# Introduction
Explain what this notebook is for.

# Initialization

In [1]:
#
# Imports
#

import requests
import io
from astropy.table import Table, unique
import numpy as np

# There are a number of relatively unimportant warnings that 
# show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")


# Utility Functions

In [38]:

# 
# Convenience functions
# 

def aptable_from_votable_response(response):
    """
    Takes a VOTABLE response from a web service and returns an astropy table.
    
    Parameters
    ----------
    response : requests.Response
        Response whose contents are assumed to be a VOTABLE.
        
    Returns
    -------
    aptable : astropy.table.Table
        Astropy Table containing the data from the first TABLE in the VOTABLE.
    """
    
    # The astropy table reader would like a file-like object, so convert
    # the response content a byte stream.  This assumes Python 3.x.
    # 
    # (The reader also accepts just a string, but that seems to have two 
    # problems:  It looks for newlines to see if the string is itself a table,
    # and we need to support unicode content.)
    file_like_content = io.BytesIO(response.content)
    
    # The astropy table reader will auto-detect that the content is a VOTABLE
    # and parse it appropriately.
    aptable = Table.read(file_like_content)
    return aptable

def tap_query_sync(query, tap_service, print_url=False):
    """
    Executes the specified TAP query at the specified TAP service and return
    the result as an astropy Table.
    """
    tap_params = {
        "request": "doQuery",
        "lang": "ADQL",
        "query": query
    }
    results = requests.post(tap_service, data=tap_params)
    if (print_url):
        print('Queried: ' + results.url)
    aptable = aptable_from_votable_response(results)
    return aptable

def sia_query(access_url, ra, dec, size, print_url=False):
    params = {
        'POS': sval(ra) + ',' + sval(dec),
        'SIZE': sval(size)
    }
    results = requests.get(access_url, params)
    if (print_url):
        print('Queried: ' + results.url)
    aptable = aptable_from_votable_response(results)
    return aptable

def sval(val):
    if (isinstance(val, bytes)):
        return str(val, 'utf-8')
    else:
        return str(val)
    
svalv = np.vectorize(sval)

def stringify_table(t):
    # This mess will look for columns that should be strings and convert them.
    scols = []
    for col in t.columns:
        colobj = t.columns[col]
        if (colobj.dtype == 'object' and isinstance(t[colobj.name][0], bytes)):
            scols.append(colobj.name)

    for colname in scols:
        t[colname] = svalv(t[colname])


# Find SIA Services
<div class="alert alert-block alert-danger">
<b>TBD:</b> Decide which columns to return from this based on helping figure out which services to use.
</div>

In [40]:
#
# Perform a synchronous TAP query on the NAVO registry to find Simple Image Access
# services from MAST.
#
tap_service_url = "https://vao.stsci.edu/RegTAP/TapService.aspx/sync"
query = """
    select b.waveband,b.short_name,a.ivoid,b.res_description,c.access_url from rr.capability a 
    natural join rr.resource b 
    natural join rr.interface c
    where a.cap_type='SimpleImageAccess' and a.ivoid like 'ivo://%stsci%' 
    order by short_name
"""
results = tap_query_sync(query, tap_service_url, print_url=True)

# Ensure that the result rows are unique and that string columns are actually strings.
unique_results = unique(results, 'ivoid')
stringify_table(unique_results)

unique_results

Queried: https://vao.stsci.edu/RegTAP/TapService.aspx/sync


waveband,short_name,ivoid,res_description,access_url
str19,str23,str38,str1285,str149
infrared#optical,BoRG,ivo://archive.stsci.edu/borg,"The Brightest of Reionizing Galaxies (BoRG) observations were first obtained in HST Cycle 17 in program GO 11700 (PI: Michele Trenti). BoRG is an ongoing pure-parallel program that obtains WFC3 imaging in four filters (F606W, F098M, F125W, F160W) on random sightlines at high Galactic latitudes (|b| &gt; 30 degrees). BoRG also assimilates data from the similar Hubble Infrared Pure Parallel Imaging Extragalactic Survey (HIPPIES) pure-parallel program (GO 11702; PI: Yan). Data from the HIPPIES program uses the F600LP filter instead of F606W.",http://archive.stsci.edu/siap/search.php?id=borg&amp;
optical,GHOSTS,ivo://archive.stsci.edu/ghosts,"The GHOSTS survey is the largest study to date of the resolved stellar populations in the outskirts of disk galaxies. The sample consists of 14 disk galaxies within 17 Mpc, whose outer disks and halos are imaged with the Hubble Space Telescope Advanced Camera for Surveys (ACS).",http://archive.stsci.edu/siap/search.php?id=ghosts&amp;
uv,GALEX,ivo://archive.stsci.edu/sia/galex,"The Galaxy Evolution Explorer (GALEX), a NASA Small Explorer mission, is performing the first all-sky, deep imaging and spectroscopic ultraviolet surveys in space. The prime goal of GALEX is to study star formation in galaxies and its evolution with time.",http://galex.stsci.edu/gxWS/siap/gxsiap.aspx?
uv,GALEX,ivo://archive.stsci.edu/siap/galex,"The Galaxy Evolution Explorer (GALEX), a NASA Small Explorer mission, is performing the first all-sky, deep imaging and spectroscopic ultraviolet surveys in space. The prime goal of GALEX is to study star formation in galaxies and its evolution with time.",http://galex.stsci.edu/gxWS/SIAP/gxSIAP.aspx?
optical#infrared#uv,HLA,ivo://archive.stsci.edu/siap/hla,"The Hubble Space Telecope Legacy Archive (HLA) was developed at the Space Telescope Science Institute to optimize the science return from HST instruments. This resource is an image service which accesses all HLA observation data. The calibrated data is fully online with several forms of access including footprint visualization, composite images, extracted spectra and source lists.","http://hla.stsci.edu/cgi-bin/hlaSIAP.cgi?imagetype=best&amp;inst=ACS,ACSGrism,WFC3,WFPC2,NICMOS,NICGRISM,COS,STIS,FOS,GHRS&amp;proprietary=false&amp;"
optical#uv,HST.ACSGGCT,ivo://mast.stsci/acsggct,"The ACSGGCT program used the HST ACS/WFC instrument to obtain uniform imaging of 65 of the nearest globular clusters to provide an extensive homogeneous dataset for a broad range of scientific investigations. An overview of the project was published by Sarajedini, A. et al. 2007, ""The ACS Survey of Galactic Globular Clusters. I. Overview and Clusters Without Previous Hubble Space Telescope Photometry"" AJ, 133, 1658.",http://archive.stsci.edu/siap/search.php?id=acsggct&amp;
optical,HST.Andromeda,ivo://mast.stsci/andromeda,"Using the Advanced Camera for Surveys on the Hubble Space Telescope, Thomas Brown (STScI) et al. obtained deep optical images reaching well below the oldest main sequence turnoff in six fields of the Andromeda Galaxy. The fields fall at four positions on the southeast minor axis, one position in the giant stellar stream, and one position on the northeast major axis. These data were obtained as part of three large observing programs (9453, 10265, 10816) designed to probe the star formation history of the stellar population in various structures of the galaxy.",http://archive.stsci.edu/siap/search.php?id=andromeda&amp;
optical#uv,HST.ANGRRR,ivo://mast.stsci/angrrr,"An archive of stellar photometry for galaxies within 3.5 Mpc, excluding Local Group galaxies, based on primary and parallel wide-filter UV and optical observations taken with HST ACS/WFC or WFPC2.The sample includes all observations taken through Dec. 2008, as well as observations taken as part of Cycle 16 Supplemental GO programs 11986

# Create a Subset (optional)

In [42]:
# Select only GALEX, HLA and VLA-FIRST.  Well, skip GALEX for now...
short_list = unique_results[np.isin(unique_results['short_name'], 
                                    ['HLA', 'VLA-FIRST'])]


# Query Those Services on a Position

In [43]:
url = short_list[0]['access_url']
ra = 185.47873
dec = 4.47365
size = 0

sia_results = sia_query(url, ra, dec, size, print_url=True)

sia_results


Queried: http://hla.stsci.edu/cgi-bin/hlaSIAP.cgi?imagetype=best&amp;inst=ACS,ACSGrism,WFC3,WFPC2,NICMOS,NICGRISM,COS,STIS,FOS,GHRS&amp;proprietary=false&amp;&POS=185.47873%2C4.47365&SIZE=0


URL,RA,DEC,Level,Target,Detector,Aperture,Spectral_Elt,NExposures,ExpTime,StartTime,Dataset,PropID,VisitNum,PI_Name,DAOCat,SEXCat,Mode,DBCreationDate,Source,Wavelength,Format,ReleaseDate,NReleaseDate,Title,naxis,scale,naxes,crpix,crval,cd_matrix,filename,regionSTCS
,deg,deg,,,,,,,s,,,,,,,,,,,0.1 nm,,,,,,deg / pix,,pix,deg,deg / pix,,
object,float64,float64,int32,object,object,object,object,int32,float64,object,object,object,object,object,object,object,object,object,object,float64,object,object,int64,object,object,object,int32,object,object,object,object,object
http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_05446_4d_wfpc2_f606w_wf,185.46908300000001,4.4770649999999996,2,NGC4303,WFPC2,PC1,F606W,2,160.0,1994-06-06 19:15:17,hst_05446_4d_wfpc2_f606w_wf,5446,4d,ILLINGWORTH,Loaded,Loaded,IMAGE,2010-02-05 12:40:40,CADC,5997.0,image/fits,1995-06-07 21:15:15,802559715,05446_4d WFPC2 F606W (combined) NGC4303,[1950 1950],[2.77778e-05],2,[975.0 975.0],[185.469083 4.477065],[-2.77778e-05 0.0 0.0 2.77778e-05],hst_05446_4d_wfpc2_f606w_wf,Polygon J2000 185.46429900 4.49677100 185.46432740 4.49669020 185.44536100 4.48985500 185.45202188 4.47148838 185.45183800 4.47142100 185.45263582 4.46925775 185.45949200 4.45066700 185.48032700 4.45830100 185.47643160 4.46887017 185.48470800 4.47181600 185.48195228 4.47950057 185.49259600 4.48321900 185.48525700 4.50409400
http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_05446_4d_wfpc2_total_wf,185.46908300000001,4.4770649999999996,2,NGC4303,WFPC2,PC1,detection,2,160.0,1994-06-06 19:15:17,hst_05446_4d_wfpc2_total_wf,5446,4d,ILLINGWORTH,Loaded,Loaded,IMAGE,2010-02-05 12:40:40,CADC,5997.0,image/fits,1995-06-07 21:15:15,802559715,05446_4d WFPC2 detection (combined) NGC4303,[1950 1950],[2.77778e-05],2,[975.0 975.0],[185.469083 4.477065],[-2.77778e-05 0.0 0.0 2.77778e-05],hst_05446_4d_wfpc2_total_wf,Polygon J2000 185.45183800 4.47142100 185.45263582 4.46925775 185.45949200 4.45066700 185.48032700 4.45830100 185.47643160 4.46887017 185.48470800 4.47181600 185.48195228 4.47950057 185.49259600 4.48321900 185.48525700 4.50409400 185.46429900 4.49677100 185.46432740 4.49669020 185.44536100 4.48985500 185.45202188 4.47148838
http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_06358_08_wfpc2_f218w_wf,185.46928399999999,4.4695410000000004,2,NGC4303,WFPC2,WF3-FIX,F218W,2,1200.0,1996-03-15 22:44:16,hst_06358_08_wfpc2_f218w_wf,6358,08,COLINA,Loaded,Loaded,IMAGE,2010-02-05 15:26:18,CADC,2203.0,image/fits,1997-03-16 13:55:12,858520512,06358_08 WFPC2 F218W (combined) NGC4303,[2100 2100],[2.77778e-05],2,[1050.0 1050.0],[185.469284 4.469541],[-2.77778e-05 0.0 0.0 2.77778e-05],hst_06358_08_wfpc2_f218w_wf,Polygon J2000 185.45635100 4.49776400 185.45631547 4.49769835 185.44693440 4.48036390 185.44650542 4.47957121 185.44581100 4.47828800 185.45574989 4.47294109 185.45150600 4.46526900 185.45867497 4.46133072 185.45317000 4.45151700 185.47253700 4.44072200 185.48337500 4.46004600 185.48330019 4.46008771 185.49297000 4.47773100 185.47577938 4.48709586 185.47587200 4.48726700
http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&amp;dataset=hst_06358_08_wfpc2_total_wf,185.46928399999999,4.4695410000000004,2,NGC4303,WFPC2,WF3-FIX,detection,2,1200.0,1996-03-15 22:44:16,hst_06358_08_wfpc2_total_wf,6358,08,COLINA,Loaded,Loaded,IMAGE,2010-02-05 15:26:18,CADC,2203.0,image/fits,1997-03-16 13:55:12,858520512,06358_08 WFPC2 detection (combined) NGC4303,[2100 2100],[2.77778e-05],2,[1050.0 1050.0],[185.469284 4.469541],[-2.77778e-05 0.0 0.0 2.77778e-05],hst_06358_08_wfpc2_total_wf,Polygon J2000 185.48330019 4.46008771 185.49297000 4.47773100 185.47577938 4.48709586 185.47587200 4.48726700 185.45635100 4.49776400 185.45631547 4.49769835 185.44693440 4.48036390 185.44650542 4.47957121 185.44581100 4.47828800 185.45574989 4.47294109 185.45150600 4.46526900 185.45867497 4.46133072 185.45317000 4.45151700 185.47253700 4.44072200 185.48337500 4.46004600
http://hla.stsci.edu/cgi-bin/getdata.cgi?conf

# Display Some Results